# Analyse statistique du dataset NSynth

Ce notebook propose une exploration statistique du dataset NSynth à partir du fichier `examples.json`.

**Plan :**
1. Chargement et aperçu du dataset
2. Distribution des sample rates
3. Analyse des qualités (`qualities` et `qualities_str`)
4. Analyse des sources d’instrument (`instrument_source`, `instrument_source_str`)
5. Analyse des familles d’instrument (`instrument_family`, `instrument_family_str`)
6. Lien entre `instrument_family` et `instrument_family_str`
7. Distribution des pitches
8. Distribution des instruments (`instrument`, `instrument_str`)
9. Distribution des vélocités
10. Analyse croisée : familles, sources, qualités, pitches, instruments

---

## 1. Chargement et aperçu du dataset

Charger le fichier JSON, afficher un échantillon, et lister les clés et champs disponibles pour chaque entrée.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Charger le fichier JSON (attention: fichier volumineux)
with open('nsynth-valid/examples.json', 'r') as f:
    data = json.load(f)

# Conversion en DataFrame
records = []
for k, v in data.items():
    entry = {'note_str': k}
    entry.update(v)
    records.append(entry)
df = pd.DataFrame(records)

df.head()

In [ ]:
# Afficher les clés disponibles pour chaque entrée
print('Champs disponibles dans une entrée :')
print(list(df.columns))

# Afficher un exemple complet
print('\nExemple d\'entrée :')
df.iloc[0].to_dict()

## 2. Distribution des sample rates

Extraire les valeurs de `sample_rate`, afficher leur distribution (histogramme, valeurs uniques, statistiques descriptives).

In [ ]:
# Distribution des sample_rate
if 'sample_rate' in df.columns:
    print('Valeurs uniques de sample_rate :', df['sample_rate'].unique())
    print(df['sample_rate'].describe())
    plt.figure(figsize=(6,3))
    sns.histplot(df['sample_rate'], bins=10, kde=False)
    plt.title('Distribution des sample rates')
    plt.xlabel('Sample rate')
    plt.ylabel('Count')
    plt.show()
else:
    print('Champ sample_rate absent dans ce dataset.')

## 3. Analyse des qualités (`qualities` et `qualities_str`)

Lister toutes les qualités possibles, compter leur occurrence, afficher la distribution des qualités (barplot, heatmap multi-label si pertinent).

In [ ]:
# Analyse des qualités
# Extraction des qualités (multi-label)
all_qualities = []
if 'qualities_str' in df.columns:
    for qlist in df['qualities_str'].dropna():
        all_qualities.extend(qlist)
    qual_counts = pd.Series(all_qualities).value_counts()
    print('Qualités possibles :', qual_counts.index.tolist())
    print(qual_counts)
    plt.figure(figsize=(8,3))
    sns.barplot(x=qual_counts.index, y=qual_counts.values)
    plt.title('Distribution des qualités (qualities_str)')
    plt.ylabel('Occurrences')
    plt.xticks(rotation=45)
    plt.show()
else:
    print('Champ qualities_str absent dans ce dataset.')

In [ ]:
# Heatmap multi-label (présence/absence de chaque qualité)
if 'qualities_str' in df.columns:
    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    qualities_matrix = mlb.fit_transform(df['qualities_str'].fillna('').apply(lambda x: x if isinstance(x, list) else []))
    qual_df = pd.DataFrame(qualities_matrix, columns=mlb.classes_)
    plt.figure(figsize=(10,6))
    sns.heatmap(qual_df.corr(), annot=True, cmap='coolwarm')
    plt.title('Corrélation entre les qualités')
    plt.show()

## 4. Analyse des sources d’instrument (`instrument_source`, `instrument_source_str`)

Compter et visualiser la distribution des sources d’instrument, faire le lien entre l’index et la chaîne de caractères.

In [ ]:
# Distribution des sources d'instrument
if 'instrument_source' in df.columns and 'instrument_source_str' in df.columns:
    src_counts = df['instrument_source_str'].value_counts()
    print('Sources d\'instrument :', src_counts.index.tolist())
    print(src_counts)
    plt.figure(figsize=(6,3))
    sns.barplot(x=src_counts.index, y=src_counts.values)
    plt.title('Distribution des sources d\'instrument')
    plt.ylabel('Occurrences')
    plt.show()
    # Lien index <-> string
    mapping = df[['instrument_source', 'instrument_source_str']].drop_duplicates().sort_values('instrument_source')
    print('Mapping index <-> string :')
    print(mapping)
else:
    print('Champs instrument_source ou instrument_source_str absents.')

## 5. Analyse des familles d’instrument (`instrument_family`, `instrument_family_str`)

Compter et visualiser la distribution des familles d’instrument, faire le lien entre l’index et la chaîne de caractères.

In [ ]:
# Distribution des familles d'instrument
if 'instrument_family' in df.columns and 'instrument_family_str' in df.columns:
    fam_counts = df['instrument_family_str'].value_counts()
    print('Familles d\'instrument :', fam_counts.index.tolist())
    print(fam_counts)
    plt.figure(figsize=(8,3))
    sns.barplot(x=fam_counts.index, y=fam_counts.values)
    plt.title('Distribution des familles d\'instrument')
    plt.ylabel('Occurrences')
    plt.xticks(rotation=45)
    plt.show()
    # Lien index <-> string
    mapping = df[['instrument_family', 'instrument_family_str']].drop_duplicates().sort_values('instrument_family')
    print('Mapping index <-> string :')
    print(mapping)
else:
    print('Champs instrument_family ou instrument_family_str absents.')

## 6. Lien entre `instrument_family` et `instrument_family_str`

Vérifier la correspondance entre les deux champs, afficher un mapping unique, détecter d’éventuelles incohérences.

In [ ]:
# Vérification de la correspondance unique
if 'instrument_family' in df.columns and 'instrument_family_str' in df.columns:
    mapping = df[['instrument_family', 'instrument_family_str']].drop_duplicates()
    print('Mapping unique instrument_family <-> instrument_family_str :')
    print(mapping)
    # Détection d'incohérences
    counts = mapping.groupby('instrument_family').size()
    if (counts > 1).any():
        print('Attention : incohérences détectées dans le mapping !')
    else:
        print('Mapping cohérent (1-1) entre index et string.')
else:
    print('Champs instrument_family ou instrument_family_str absents.')

## 7. Distribution des pitches

Extraire les valeurs de `pitch`, afficher leur distribution (histogramme, statistiques descriptives, min/max/moyenne).

In [ ]:
# Distribution des pitches
if 'pitch' in df.columns:
    print(df['pitch'].describe())
    plt.figure(figsize=(10,4))
    sns.histplot(df['pitch'], bins=60, kde=True)
    plt.title('Distribution des pitches')
    plt.xlabel('Pitch (MIDI)')
    plt.ylabel('Count')
    plt.show()
else:
    print('Champ pitch absent dans ce dataset.')

## 8. Distribution des instruments (`instrument`, `instrument_str`)

Compter les occurrences de chaque instrument, faire le lien entre l’index et la chaîne, visualiser les instruments les plus fréquents.

In [ ]:
# Distribution des instruments
if 'instrument' in df.columns and 'instrument_str' in df.columns:
    instr_counts = df['instrument_str'].value_counts().head(20)
    print('Instruments les plus fréquents :')
    print(instr_counts)
    plt.figure(figsize=(10,4))
    sns.barplot(x=instr_counts.index, y=instr_counts.values)
    plt.title('Top 20 instruments les plus fréquents')
    plt.ylabel('Occurrences')
    plt.xticks(rotation=90)
    plt.show()
    # Lien index <-> string
    mapping = df[['instrument', 'instrument_str']].drop_duplicates().sort_values('instrument')
    print('Mapping index <-> string (extrait) :')
    print(mapping.head(10))
else:
    print('Champs instrument ou instrument_str absents.')

## 9. Distribution des vélocités

Extraire les valeurs de `velocity`, afficher leur distribution (histogramme, statistiques descriptives).

In [ ]:
# Distribution des vélocités
if 'velocity' in df.columns:
    print(df['velocity'].describe())
    plt.figure(figsize=(8,3))
    sns.histplot(df['velocity'], bins=32, kde=True)
    plt.title('Distribution des vélocités')
    plt.xlabel('Velocity')
    plt.ylabel('Count')
    plt.show()
else:
    print('Champ velocity absent dans ce dataset.')

## 10. Analyse croisée : familles, sources, qualités, pitches, instruments

Croiser les variables principales (famille, source, qualité, pitch, instrument) pour explorer les corrélations et tendances (tableaux croisés, heatmaps, pairplots).

In [ ]:
# Tableau croisé familles x sources
if 'instrument_family_str' in df.columns and 'instrument_source_str' in df.columns:
    crosstab = pd.crosstab(df['instrument_family_str'], df['instrument_source_str'])
    print(crosstab)
    plt.figure(figsize=(8,6))
    sns.heatmap(crosstab, annot=True, fmt='d', cmap='Blues')
    plt.title('Famille d\'instrument vs Source')
    plt.ylabel('Famille')
    plt.xlabel('Source')
    plt.show()

# Pairplot sur pitch, velocity, instrument_family
sns.pairplot(df, vars=['pitch', 'velocity', 'instrument_family'], hue='instrument_family_str', corner=True)
plt.suptitle('Pairplot : pitch, velocity, famille', y=1.02)
plt.show()